In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import warnings 
warnings.filterwarnings('ignore')

# Import the Libraries

In [ ]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import math
import xgboost as xgb


from scipy.stats import skew
from scipy import stats

from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, auc, classification_report, roc_curve, roc_auc_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

from datetime import datetime

%matplotlib inline
print('done')



# Import Data

In [ ]:
train = pd.read_csv('../input/funding-successful-projects/train.csv')
test = pd.read_csv('../input/funding-successful-projects/test.csv')

# Explore the Data

In [ ]:
train.head(5)

In [ ]:
train.describe()

In [ ]:
g = sns.heatmap(train[['final_status', 'goal', 'country', 'currency', 'backers_count', 'disable_communication']].corr(), cmap = 'coolwarm')

In [ ]:
fig = plt.figure(figsize=(20,1))
sns.countplot(y='final_status', data=train)
train.final_status.value_counts()

In [ ]:
# Missing Values
train.isnull().sum()

### Goal vs Final Status

In [ ]:
def comparing(train, variable1, variable2):
    print(train[[variable1, variable2]][train[variable2].isnull() == False].groupby([variable1], as_index=False).mean().sort_values(by=variable2, ascending=False))
    g = sns.FacetGrid(train, col=variable2).map(sns.distplot, variable1)

In [ ]:
def counting_values(train, variable1, variable2):
    return train[[variable1, variable2]][train[variable2].isnull()==False].groupby([variable1], as_index=False).mean().sort_values(by=variable2, ascending=False)

In [ ]:
comparing(train, 'goal', 'final_status')

In [ ]:
train['log_goal'] = np.log10(train['goal'])

In [ ]:
train.head(5)

### log_goal vs Final Status

In [ ]:
comparing(train, 'log_goal', 'final_status')

In [ ]:
fig, axs = plt.subplots(figsize=(10,5))
sns.histplot(data=train, x='log_goal').set_title('Log_Goal Distribution', fontdict={'fontsize':20
                                                                                   , 'fontweight':'bold'});
sns.despine()

In [ ]:
facet = sns.FacetGrid(train, hue='final_status', aspect=3)
facet.map(sns.kdeplot, 'log_goal', shade=True)
facet.set(xlim=(0, train['log_goal'].max()))
facet.add_legend()

In [ ]:
sns.lmplot(x = 'log_goal', y = 'backers_count', col ='final_status', data = train, fit_reg = False)

In [ ]:
sns.lmplot(x = 'log_goal', y = 'backers_count', data = train[train['final_status'] == 0], fit_reg = False)
plt.title('Unsuccesful Kickstarters rarely find backers with goals above $1 Million')

In [ ]:
sns.lmplot(x = 'log_goal', y = 'backers_count', data = train[train['final_status'] == 1], fit_reg = False)
plt.title('Successfull Kickstarters')

In [ ]:
#  Projects with goal over $ 1Million
train[train['goal']>1000000].shape 

In [ ]:
# Funded projects with over $1Million goal
train[train['goal']>1000000]['final_status'].value_counts() 

In [ ]:
#percent of projects funded with goal over $1M
percent_over_1M = (4/232)*100
percent_over_1M

In [ ]:
#  Projects with goal < $ 1Million
train[train['goal']<1000000].shape 

In [ ]:
# Funded projects with < $1Million goal
train[train['goal']<1000000]['final_status'].value_counts() 

In [ ]:
percent = (34553/107811)*100
percent

#### Goals less than $1M got funded

In [ ]:
plt.figure(figsize = (6,6))
sns.boxplot(x ='final_status', y = 'log_goal', data = train)
plt.title('Successful and Unsuccessful Kickstarters average goal comparision')


In [ ]:
#train.loc[train['log_goal'] <= 5 , 'backers_count'] = 1
#train.loc[train['log_goal'] > 5 , 'backers_count'] = 0

In [ ]:
train.loc[train['log_goal'] <= 1  , 'log_goal'] = 0 
train.loc[(train['log_goal'] >1 ) & (train['log_goal']<= 3) , 'log_goal'] = 1  
train.loc[train['log_goal'] > 3 & (train['log_goal']<=5), 'log_goal'] = 2  
train.loc[train['log_goal'] > 5 , 'log_goal'] = 3  

In [ ]:
train.head(5)

### Country vs Final_status

In [ ]:
country = train.groupby('country')['final_status'].sum()
country

In [ ]:
fig, axs = plt.subplots(figsize=(10,5))
sns.histplot(data=train, x='country', hue='final_status').set_title('Country Distribution', fontdict={'fontsize':20
                                                                                   , 'fontweight':'bold'});
sns.despine()

In [ ]:
sns.countplot(data=train, x = 'country', hue='final_status')

### Currency vs Final Status

In [ ]:
currency = train.groupby('currency')['final_status'].sum()
currency

In [ ]:
sns.countplot(data=train, x = 'currency', hue='final_status')

### Name vs Final Status

In [ ]:
train['NameLen'] = train['name'].str.len()
train.head(5)

In [ ]:
fig, axs = plt.subplots(figsize=(10,5))
sns.set_palette('Set1')
sns.histplot(data=train, x='NameLen', hue='final_status').set_title('NameLen Distribution', fontdict={'fontsize':20
                                                                                   , 'fontweight':'bold'});
sns.despine()

In [ ]:
sns.countplot(data=train, x='NameLen', hue='final_status')

Bins
1. 0-150
2. 150-250
3. 250-400
4. greater than 400


In [ ]:
#NameLen bins
train.loc[train['NameLen'] > 0 & (train['NameLen'] <= 150) , 'NameLen'] = 0 
train.loc[(train['NameLen'] > 150) & (train['NameLen']<= 250) , 'NameLen'] = 1  
train.loc[train['NameLen'] > 250 & (train['NameLen']<=400), 'NameLen'] = 2  
train.loc[train['NameLen'] > 400 , 'NameLen'] = 3  

### Keyword vs Final Status

In [ ]:
train.keywords.unique()[0]

In [ ]:
train.keywords.unique()[2021]

In [ ]:
train['keywordsLen'] = train['keywords'].str.len()
train

In [ ]:
fig, axs = plt.subplots(figsize=(10,5))
sns.set_palette('Set1')
sns.histplot(data=train, x='keywordsLen', hue='final_status').set_title('KeywordsLen Distribution', fontdict={'fontsize':20
                                                                                   , 'fontweight':'bold'});
sns.despine()

In [ ]:
train.keywordsLen.describe()

### Desc vs Final Status

In [ ]:
train['descLen'] = train['desc'].str.len()
train.head(5)

In [ ]:
fig, axs = plt.subplots(figsize=(10,5))
sns.set_palette('Set1')
sns.histplot(data=train, x='descLen', hue='final_status').set_title('DescLen Distribution', fontdict={'fontsize':20
                                                                                   , 'fontweight':'bold'});
sns.despine()

In [ ]:
train.descLen.describe()

In [ ]:
sns.displot(x='descLen', hue='final_status', data=train)

Bins
1. 0-200
2. 200-300
3. 300-400
4. greater than 400

In [ ]:
#descLen bins    
train.loc[train['descLen'] > 0 & (train['descLen'] <= 200) , 'descLen'] = 0 
train.loc[(train['descLen'] > 200) & (train['descLen']<= 300) , 'descLen'] = 1  
train.loc[train['descLen'] > 300 & (train['descLen']<=400), 'descLen'] = 2  
train.loc[train['descLen'] > 400 , 'descLen'] = 3  

### Time duration to launch

In [ ]:
# Convert UNIX time format to Standard time format
date_time = ['deadline', 'state_changed_at', 'created_at', 'launched_at']
for i in date_time:
    train[i] = train[i].apply(lambda x: datetime.fromtimestamp(int(x)).strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
train.head(5)

In [ ]:
# Duration in weeks
def Duration(d):
    launch = datetime.strptime(d[0], '%Y-%m-%d %H:%M:%S')
    deadline = datetime.strptime(d[1], '%Y-%m-%d %H:%M:%S')
    duration = deadline-launch
    weekDifference = int(duration.total_seconds()/(3600*24*7))
    return weekDifference
train['duration'] = train[['launched_at', 'deadline']].apply(lambda d: Duration(d), axis=1)

In [ ]:
train.head(5)

In [ ]:
train.duration.describe()

In [ ]:
fig, axs = plt.subplots(figsize=(10,5))
sns.set_palette('Set1')
sns.histplot(data=train, x='duration').set_title('Duration Distribution', fontdict={'fontsize':20
                                                                                   , 'fontweight':'bold'});
sns.despine()

In [ ]:
auxlog_goal = pd.cut(train['duration'], 8)
fig, axs = plt.subplots(figsize=(15,5))
sns.countplot(x=auxlog_goal, hue='final_status', data=train).set_title('Duration Bins', fontdict={'fontsize':20
                                                                                                 , 'fontweight': 'bold'})
sns.despine()

In [ ]:
#Duration bins   
train.loc[train['duration'] >= 2 , 'duration'] = 0
train.loc[train['duration'] > 2 & (train['duration'] <= 4) , 'duration'] = 1  
train.loc[train['duration'] > 4 & (train['duration']<= 6) , 'duration'] = 2  
train.loc[train['duration'] > 6 & (train['duration']<=8), 'duration'] = 3   
train.loc[train['duration'] > 8 , 'duration'] = 4   

### Disable Communication vs Final Status

In [ ]:
train.disable_communication.describe()

In [ ]:
sns.countplot(data = train, x = 'disable_communication', hue= 'final_status')

In [ ]:
train.head(5)

Features 
1. NameLen 
2. descLen
3. log_goal
4. country
5. currency
6. duration
7. backers_count  X
8. disable_communication  X

# Test Data

In [ ]:
# log_goal
test['log_goal'] = np.log10(test['goal'])

In [ ]:
test.insert(loc=12, column='backers_count', value=0)

In [ ]:
#test.loc[test['log_goal'] <= 5 , 'backers_count'] = 1
#test.loc[test['log_goal'] > 5 , 'backers_count'] = 0
    

In [ ]:
test.loc[test['log_goal'] <= 1  , 'log_goal'] = 0 
test.loc[test['log_goal'] >1  & (test['log_goal']<= 3) , 'log_goal'] = 1  
test.loc[test['log_goal'] > 3 & (test['log_goal']<=5), 'log_goal'] = 2  
test.loc[test['log_goal'] > 5 , 'log_goal'] = 3  

In [ ]:
test.head(5)

In [ ]:
# Name length
test['NameLen'] = test['name'].str.len()

In [ ]:
#NameLen bins
test.loc[test['NameLen'] > 0 & (test['NameLen'] <= 150) , 'NameLen'] = 0 
test.loc[(test['NameLen'] > 150) & (test['NameLen']<= 250) , 'NameLen'] = 1  
test.loc[test['NameLen'] > 250 & (test['NameLen']<=400), 'NameLen'] = 2  
test.loc[test['NameLen'] > 400 , 'NameLen'] = 3  

In [ ]:
# keywords length
test['keywordsLen'] = test['keywords'].str.len()

In [ ]:
#desc length
test['descLen'] = test['desc'].str.len()

In [ ]:
#descLen bins     
test.loc[test['descLen'] > 0 & (test['descLen'] <= 200) , 'descLen'] = 0  
test.loc[test['descLen'] > 200 & (test['descLen']<= 300) , 'descLen'] = 1  
test.loc[test['descLen'] > 300 & (test['descLen']<=400), 'descLen'] = 2  
test.loc[test['descLen'] > 400, 'descLen'] = 3 

In [ ]:
# Convert UNIX time format to Standard time format
date_time = ['deadline', 'state_changed_at', 'created_at', 'launched_at']
for i in date_time:
    test[i] = test[i].apply(lambda x: datetime.fromtimestamp(int(x)).strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
# Duration in weeks

test['duration'] = test[['launched_at', 'deadline']].apply(lambda d: Duration(d), axis=1)

In [ ]:
#Duration bins   
test.loc[test['duration'] >= 2 , 'duration'] = 0
test.loc[test['duration'] > 2 & (test['duration'] <= 4) , 'duration'] = 1  
test.loc[test['duration'] > 4 & (test['duration']<= 6) , 'duration'] = 2  
test.loc[test['duration'] > 6 & (test['duration']<=8), 'duration'] = 3   
test.loc[test['duration'] > 8 , 'duration'] = 4   

In [ ]:
test.info()

In [ ]:
test.log_goal.astype(int)

# Model Selection

In [ ]:
my_features = ['NameLen', 'keywordsLen','descLen', 'backers_count', 'country', 'currency', 'duration', 'log_goal']

In [ ]:
dtrain = train[my_features].copy()
dtest = test[my_features].copy()

In [ ]:
y = train.final_status
df_train = dtrain
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
X_train, X_valid, y_train, y_valid = train_test_split(train, y, test_size=0.2, random_state=0)
categorical_cols = [cname for cname in df_train.columns if
                    df_train[cname].nunique() < 10 and 
                    df_train[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in df_train.columns if 
                df_train[cname].dtype in ['int64', 'float64']]

# Preprocessing

In [ ]:
# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='mean')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

#Feature Scaling
sc = StandardScaler()

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [ ]:
Results = pd.DataFrame({'Model': [], 'Accuracy Score': []})

## LogisticRegression

In [ ]:
# Define model
lrc = LogisticRegression()


# Bundle preprocessing and modeling code in a pipeline
lr = Pipeline(steps=[('preprocessor', preprocessor),
                      ('sc',StandardScaler()),
                      ('model', lrc)
])

# Preprocessing of training data, fit model 
lr.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
predsLR = lr.predict(X_valid)

print('Accuracy:', accuracy_score(y_valid, predsLR))
print('CR:', classification_report(y_valid, predsLR))
print('CM:',confusion_matrix(y_valid, predsLR))

res = pd.DataFrame({'Model': ['LogisticRegression'],
                    'Accuracy Score': [accuracy_score(y_valid, predsLR)]})
Results = Results.append(res)

## DecisionTree

In [ ]:
# Define model
dtc = DecisionTreeClassifier()


# Bundle preprocessing and modeling code in a pipeline
dt = Pipeline(steps=[('preprocessor', preprocessor),
                      ('sc',StandardScaler()),
                      ('model', dtc)
])

# Preprocessing of training data, fit model 
dt.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
predsDT = dt.predict(X_valid)

print('Accuracy:', accuracy_score(y_valid, predsDT))
print('CR:', classification_report(y_valid, predsDT))
print('CM:',confusion_matrix(y_valid, predsDT))

res = pd.DataFrame({'Model': ['DecisionTree'],
                    'Accuracy Score': [accuracy_score(y_valid, predsDT)]})
Results = Results.append(res)

## RandomForest

In [ ]:
# Define model
model = RandomForestClassifier(n_estimators=100, random_state=0,criterion='entropy')

# Bundle preprocessing and modeling code in a pipeline
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('sc',StandardScaler()),
                      ('model', model)
])

# Preprocessing of training data, fit model 
clf.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = clf.predict(X_valid)
print('Accuracy:', accuracy_score(y_valid, preds))
print('CR:', classification_report(y_valid, preds))
print('CM:',confusion_matrix(y_valid, preds))

res = pd.DataFrame({'Model': ['RandomForest'],
                    'Accuracy Score': [accuracy_score(y_valid, preds)]})
Results = Results.append(res)

## KNN

In [ ]:
# Define model
knnc = KNeighborsClassifier()


# Bundle preprocessing and modeling code in a pipeline
knn = Pipeline(steps=[('preprocessor', preprocessor),
                      ('sc',StandardScaler()),
                      ('model', knnc)
])

# Preprocessing of training data, fit model 
knn.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
predsKNN = knn.predict(X_valid)

print('Accuracy:', accuracy_score(y_valid, predsKNN))
print('CR:', classification_report(y_valid, predsKNN))
print('CM:',confusion_matrix(y_valid, predsKNN))

res = pd.DataFrame({'Model': ['KNN'],
                    'Accuracy Score': [accuracy_score(y_valid, predsKNN)]})
Results = Results.append(res)

## SVM

In [ ]:
# Define model
sv = SVC()


# Bundle preprocessing and modeling code in a pipeline
svc = Pipeline(steps=[('preprocessor', preprocessor),
                      ('sc',StandardScaler()),
                      ('model', sv)
])

# Preprocessing of training data, fit model 
svc.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
predssvc = svc.predict(X_valid)

print('Accuracy:', accuracy_score(y_valid, predssvc))
print('CR:', classification_report(y_valid, predssvc))
print('CM:',confusion_matrix(y_valid, predssvc))

res = pd.DataFrame({'Model': ['SVM'],
                    'Accuracy Score': [accuracy_score(y_valid, predssvc)]})
Results = Results.append(res)

## XGB

In [ ]:
from xgboost import XGBClassifier
model_xgb = XGBClassifier(n_estimators= 200, learning_rate = 0.1, max_depth=2, min_child_weight = 1,nthread=4, seed=27,subsample=0.8,colsample_bytree=0.9,max_delta_step=0,
                         objective= 'multi:softmax',gamma = 0,reg_alpha=0.001,reg_lambda=0.5, eval_metric='auc',random_state=0, num_class= 6)
# Bundle preprocessing and modeling code in a pipeline
xgb = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model_xgb', model_xgb)
])

# Preprocessing of training data, fit model 
xgb.fit(X_train, y_train)


In [ ]:
# Preprocessing of validation data, get predictions
predictions = xgb.predict(X_valid)
print('Accuracy:', accuracy_score(y_valid, predictions))
print('CR:', classification_report(y_valid, predictions))
print('CM:',confusion_matrix(y_valid, predictions))

res = pd.DataFrame({'Model': ['XGB'],
                    'Accuracy Score': [accuracy_score(y_valid, predictions)]})
Results = Results.append(res)

# Results

In [ ]:
Results

References:
1. https://www.kaggle.com/phuongpm/kickstarter-funding
2. https://www.kaggle.com/tkkanno/what-goal-should-i-set-for-my-kickstarter